In [2]:
from pykrige.ok import OrdinaryKriging
# from pykrige.uk import UniversalKriging
# from pykrige.rk import RegressionKriging
import geopandas as gpd
import rasterio
import rioxarray
import xarray

In [4]:
# load the bathy points

pts_gdf_all = gpd.read_file('../data/resample_test/thinned.shp',mode='r')
# pts_gdf = pts_gdf.drop(columns=['fema_elev']).dropna()

In [34]:
# keep points within a half meter of the estimated seafloor
# pts_gdf = pts_gdf[abs(pts_gdf.Z_g - pts_gdf.z_kde) < 0.005]

In [5]:
pts_gdf = pts_gdf_all.sample(5000)

In [6]:
# pts_gdf = pts_gdf.sort_values('kde_val',ascending=False).iloc[:3000]

In [7]:
ras = rasterio.open('../data/resample_test/bilinear.tif')

In [8]:
ar = rioxarray.open_rasterio(ras)
gridx = ar.x.data
gridy = ar.y.data

In [10]:
# going to randomly downsample the lidar point dataframe to make the UK model faster

# pts_gdf = pts_gdf.sample(1000)
pts_gdf.to_file('../data/resample_test/keys_testpts.gpkg')

x_loc = pts_gdf.geometry.x.to_numpy()
y_loc = pts_gdf.geometry.y.to_numpy()
z_elev = pts_gdf.Mean.to_numpy()

/home/mlinds/mambaforge/envs/thesis_env/lib/python3.9/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [11]:
okmodel = OrdinaryKriging(
    x=x_loc,
    y=y_loc,
    z=z_elev,
    variogram_model='linear',
    verbose=True,
    coordinates_type = "euclidean",
)

# ukmodel = UniversalKriging(
#     x=x_loc,
#     y=y_loc,
#     z=z_elev,
#     variogram_model='linear',
#     verbose=True,
#     # coordinates_type = "euclidean",
# )
    

Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'linear' Variogram Model
Slope: 8.252191348406586e-05
Nugget: 14.692171269129194 

Calculating statistics on variogram model fit...


In [12]:
z,ss = okmodel.execute('grid',gridx,gridy)

Executing Ordinary Kriging...



In [43]:
ras.dtypes

('float32',)

In [13]:
# xarray and rioxarray are too fuckin complicated, going to just use rasterio if I can

with rasterio.open('../data/resample_test/interp.tif',mode='w+',crs=ras.crs,width=ras.width,height=ras.height,count=2,dtype=ras.dtypes[0],transform=ras.transform,) as rasout:
    rasout.write(z,1)
    rasout.write(ss,2)


In [45]:
masks= ras.read_masks(1)